<h1 align="center">Integrating Local Music Library to Spotify Playlists</h1>
<br>
As of late May 2020, per <b>Spotify</b> documentation, <i>*it is not currently possible to add local files to playlists using the Web API...</i>". The best alternative therefore is to use the Web API to query the available equivalent tracks of local files and add those to a new or existing playlist. When no equivalent is available, the user is prompted to add the local files to the playlist manually, or skip the missing tracks.

### Usage ###
Tracks are fetched by supplying the source folder. The *artist* and *title* are queried from the ID tags (MP3 and FLAC audio files are supported). If no tags are available, the artist-track parsing is done at the the filename level by indicating the delimiter (*sep*). These information are used to perform a detailed search request to the Web API and the best-match track is retrieved. The tracks are scored using string-similarity &ndash; *Spotify's artists and title vs. local tag's artists and title* &ndash; and the user identifies the accept-fail score threshold. 

All information (including filename, artist tag, title tag, equivalent Spotify track, Spotify URI, similarity score, etc.) are stored in a Pandas dataframe. User is free to review, update, or delete entries before appending the tracks to a new or existing playlist.

### Example ###

The details of the music files in the given path are populated in the *df* using **extract_local_files**.

In [1]:
df = extract_local_files(path, " - ", False, "MP3", "flac")
display(df)

,time,filename,local_artist,local_title
0,1.324313e+09,The Shins - Caring Is Creepy.flac,The Shins,Caring Is Creepy
1,1.332749e+09,The BPA feat. Emmy the Great - Seattle.flac,The BPA feat. Emmy the Great,Seattle
2,1.363290e+09,Pinback - Proceed to Memory.flac,Pinback,Proceed to Memory
3,1.368451e+09,RAC feat. Penguin Prison - Hollywood.mp3,RAC feat. Penguin Prison,Hollywood
4,1.382070e+09,Passion Pit - Constant Conversations.mp3,Passion Pit,Constant Conversations
5,1.387264e+09,TOKiMONSTA feat. MNDR - Go With It (CRNKN Remi...,TOKiMONSTA,Go With It feat. MNDR (CRNKN Remix)
6,1.558166e+09,Vampire Weekend feat. Haim - This Life (Live V...,Vampire Weekend feat. Haim,This Life (Live Version)
7,1.590975e+09,Sylvan Esso ft. Rostam - PARAD(wm)E [Rostam Re...,Sylvan Esso feat. Rostam,PARAD(w/m)E (Rostam Remix)


The **scrape_spotify** function updates the *df* by adding the details of the closest available Spotify equivalent for each track. The details include the *artists*, *title*, the track *URI*, and the *score* of how similar the best match track is to the local file. This similarity is based on the artist and title from the IDv3 tag or from the filename.

The Emmy the Great track did not fetch results and have a score of 0, while the Pinback track perfectly matched the details of the local file and thus had a score of 1.0. In this case, the missing track happened to be unavailable in the immediate market of the user (US).

In [2]:
df = scrape_spotify(df)
display(df)

No matches for The BPA feat. Emmy the Great - Seattle.



,time,filename,local_artist,local_title,artists,title,uri,score
0,1.324313e+09,The Shins - Caring Is Creepy.flac,The Shins,Caring Is Creepy,The Shins,Caring Is Creepy,spotify:track:2AZmlhK1ssultfpYk3EIEH,1.000000
1,1.332749e+09,The BPA feat. Emmy the Great - Seattle.flac,The BPA feat. Emmy the Great,Seattle,,,,0.000000
2,1.363290e+09,Pinback - Proceed to Memory.flac,Pinback,Proceed to Memory,Pinback,Proceed to Memory,spotify:track:0W0y4CzLCy4y7pIBOoZ2ZC,1.000000
3,1.368451e+09,RAC feat. Penguin Prison - Hollywood.mp3,RAC feat. Penguin Prison,Hollywood,RAC Penguin Prison,Hollywood,spotify:track:3xEqegrcyQtfhWeEMnDyCG,0.923008
4,1.382070e+09,Passion Pit - Constant Conversations.mp3,Passion Pit,Constant Conversations,Passion Pit,Constant Conversations,spotify:track:3p9x4bK5bwmgXE6POMvOp0,1.000000
5,1.387264e+09,TOKiMONSTA feat. MNDR - Go With It (CRNKN Remi...,TOKiMONSTA,Go With It feat. MNDR (CRNKN Remix),TOKiMONSTA MNDR,Go With It,spotify:track:5dFhRQKsHnAiyg9wXKtJDz,0.855111
6,1.558166e+09,Vampire Weekend feat. Haim - This Life (Live V...,Vampire Weekend feat. Haim,This Life (Live Version),Vampire Weekend,This Life,spotify:track:4dRqYKhLVujxiBXcq50YzG,0.862667
7,1.590975e+09,Sylvan Esso ft. Rostam - PARAD(wm)E [Rostam Re...,Sylvan Esso feat. Rostam,PARAD(w/m)E (Rostam Remix),Sylvan Esso Rostam,PARAD(w/m)E - Rostam Remix,spotify:track:2gG1x8gdCy3spQMJh05tic,0.902338


The **add_to_playlist** function adds each track on the *df* to the selected user playlist. Only tracks that have a *score* that is equal or greater than the threshold are added.

When a track is not available &ndash; i.e. the *artists* and *title* are empty &ndash; the user is prompted to either add the track to the playlist manually or to skip it by hitting Return. Spotify allows local tracks to be integrated to the user's song library and even allow them __[to be downloaded onto mobile devices](https://support.spotify.com/us/using_spotify/features/listen-to-local-files/)__.

In [3]:
add_to_playlist(df, threshold=0.1)

Enter index of playlist:
0 - Trance + House
1 - Alternative Folk
2 - Odd Guilty Pleasures 
3 - 2010's Indie
4 - All Things Pop
5 - SEA Rising
6 - Life in Podcasts

3

Please add The BPA feat. Emmy the Great - Seattle manually then hit Return key to proceed.


Alternatively, the user may also create a new playlist for the *df* tracks using **generate_playlist**.

In [4]:
generate_playlist(df, "My Local Files", threshold=0.6)

### API Reference ###

**strip_extension**(*file*)<br>
Removes the file extension of a given filename string.

In [5]:
strip_extension("Portugal. The Man - Atomic Man.mp3")

'Portugal. The Man - Atomic Man'

**has_ID**(*file*)<br>
Checks if file has IDv3 tag. Currently only supports *.mp3* and *.flac* files. Returns *True* or *False*.

In [6]:
has_ID(r"Music\Portugal. The Man - Atomic Man.mp3")

True


**get_artist**(*path*, *track*, *sep*, *skiptag*)<br>
If *skiptag* is *True*, retrieves artist from the filename given that it follows the format: *artist sep title*. Otherwise, retrieves artist from the IDv3 tag.

In [7]:
get_artist(r"Music", "Portugal. The Man - Atomic Man.mp3", " - ", False)

Portugal. The Man


**get_title**(*path*, *track*, *sep*, *skiptag*)<br>
If *skiptag* is *True*, retrieves title from the filename given that it follows the format: *artist sep title*. Otherwise, retrieves title from the IDv3 tag.

In [8]:
get_title(r"Music", "Portugal. The Man - Atomic Man.mp3", " - ", False)

Atomic Man


**get_keywords**(*query*, *type*="artist")<br>
If *type* is *artist*, each artist &ndash; delimited by *comma*, *ampersand*, *feat.*, or *ft.* &ndash; in the *query* string is tokenized in the returned list. If *type* is *title*, each word in the *query* string is tokenized in the returned list.

In [9]:
get_keywords("Peter, Bjorn & John feat. Victoria Bergsman", type="artist")

['peter bjorn  john victoria bergsman', 'peter', 'bjorn', 'john', 'victoria bergsman']


In [10]:
get_keywords("Young Folks (Acoustic Version)", type="title")

['young folks (acoustic version)', 'young', 'folks', 'acoustic', 'version']


**get_keys**(*artist*, *title*)<br>
Generates tuple of search terms for *artist* and *title*.

In [11]:
get_keys("Portugal. The Man", "Atomic Man")

(['portugal. the man', 'portugal. the man'], ['atomic', 'man', 'atomic man'])

**get_q**(*artist_keys*, *title_keys*)<br>
Retrieves the query results of a detailed Spotify search with *artist* and *track* labels given the *artist_keys* and *title_keys* keywords.

In the case below, it is submitting a query as such: `q=artist:strfkr%20track:jupiter&type=track`.

In [12]:
get_q(["strfkr"], ["jupiter"])

[{'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2Tz1DTzVJ5Gyh8ZwVr6ekU'},
     'href': 'https://api.spotify.com/v1/artists/2Tz1DTzVJ5Gyh8ZwVr6ekU',
     'id': '2Tz1DTzVJ5Gyh8ZwVr6ekU',...


**get_q_simple**(*artist_keys*, *title_keys*)<br>
Retrieves the query results of a simple Spotify search (i.e. without specifying which tokens are for *artist* nor which tokens are for *track*) given the *artist_keys* and *title_keys* as tagless keywords.

In the case below, it is submitting a query as such: `q=strfkr%20jupiter&type=track`.

In [13]:
get_q_simple(["strfkr"], ["jupiter"])

[{'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2Tz1DTzVJ5Gyh8ZwVr6ekU'},
     'href': 'https://api.spotify.com/v1/artists/2Tz1DTzVJ5Gyh8ZwVr6ekU',
     'id': '2Tz1DTzVJ5Gyh8ZwVr6ekU',...


**get_string_similarity**(*string1*, *string2*)<br>
Uses Jaro-Winkler to calculate similarity score of two strings.

In [14]:
get_string_similarity("Iron & Wine", "Iron and Wine")

0.89566


**get_best_match**(*artist*, *title*)<br>
Retrieves Spotify track *artist*, *title*, *URI*, and similarity *score* of closest match to given *artist* and *title* strings. Returns dictionary if an output is produced. Otherwise, returns dictionary with empty keys.

First performs detailed search using every tokenized title permutation &ndash; the *artist* along with, iteratively: `lessons in love`, `lessons`, `in`, `love`. Selects the closest match (i.e. best score) among the results.

If query returns nothing, performs detailed search using every artist permutation &ndash; the *title* along with, iteratively: `kaskade neon trees`, `kaskade`, `neon trees`. Selects the closest match (i.e. best score) among the results.

If nothing still, performs tagless search using every artist and title keywords &ndash; `kaskade neon trees lessons in love`. Selects the closest match (i.e. best score) among the results.

In [15]:
get_best_match("Kaskade feat. Neon Trees", "Lessons in Love")

{'artist': 'Kaskade Neon Trees',
 'title': 'Lessons In Love',
 'uri': 'spotify:track:3fPPBbpHUpUnZOU9QwBiWp',
 'score': 0.8965034965034965}

**extract_local_files**(*path*, *sep*=" - ", *skiptag*=False, &ast;*filetypes*)<br>
Gets information of the files of given *filetypes* in the specified *path*. If *skiptag* is *True*, retrieves information from the filename as delimited by *sep* given that it follows the format: *artist sep title*. Otherwise, retrieves information from the IDv3 tag. Returns dataframe.

In [16]:
extract_local_files(path, " - ", False, "MP3", "flac")

,time,filename,local_artist,local_title
0,1.324313e+09,The Shins - Caring Is Creepy.flac,The Shins,Caring Is Creepy


**scrape_spotify**(*df*)<br>
Updates *df* with the Spotify track *artist*, *title*, *URI*, and similarity *score* of closest match to each dataframe entry.

In [17]:
scrape_spotify(df)

,time,filename,local_artist,local_title,artists,title,uri,score
0,1.324313e+09,The Shins - Caring Is Creepy.flac,The Shins,Caring Is Creepy,The Shins,Caring Is Creepy,spotify:track:2AZmlhK1ssultfpYk3EIEH,1.0


**add_tracks**(*df*, *playlist_uri*, *threshold*)<br>
Adds tracks from *df* that have a score equal or greater than the *threshold* to the *playlist_uri* user playlist.

In [18]:
add_tracks(df, "spotify:playlist:iIfR7l...", 0.6)

**add_to_playlist**(*df*, *threshold*=0.5)<br>
Adds tracks from *df* that have a score equal or greater than the *threshold* to the user playlist selected after the prompt.

In [19]:
add_to_playlist(df, threshold=0.6)

Enter index of playlist:
0 - Best of 2010's
1 - Best of 2000's
2 - Classic Stuff


**generate_playlist**(*df*, *playlist_name*, *threshold*=0.5)<br>
Adds tracks from *df* that have a score equal or greater than the *threshold* to a new user playlist entitled *playlist_name*.

In [20]:
generate_playlist(df, "Pre-Spotify Music", threshold=0.6)